In [1]:
%reset -f
%load_ext autoreload
%autoreload 2
# import sys
# sys.path.append('/home/saptarshi/Dhruv/Dissertation/ICC/')
from generate_attributions import *
from utils import CONFIG,evaluate_exp,PREFIX,aggregate_auc_or_sum

/home/saptarshi/anaconda3/envs/Dhruv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## German Credit Dataset

In [2]:
config = CONFIG['german']
mlp_path = PREFIX + 'models/german_mlp/german_mlp_seed_1/logs/version_2/checkpoints/epoch=8-step=108.ckpt'
scm_path = PREFIX + 'models/german_nf/german_nf_seed_2/logs/version_0/checkpoints/epoch=131-step=1584.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [3]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [4]:
# Load JSON
with open(PREFIX + "attributions/german/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

In [5]:
ge_dict = data["ge_dict"]
evaluation_metrics,evaluation_metrics_per_sample = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)


{'shap': {'pgi': {'k=1': [0.19491274654865265], 'k=2': [0.19204384088516235], 'k=3': [0.2049812376499176], 'k=4': [0.21626795828342438]}, 'pgu': {'k=1': [0.1549122929573059], 'k=2': [0.15142714977264404], 'k=3': [0.12340263277292252], 'k=4': [1.3307929869199597e-08]}}, 'lime': {'pgi': {'k=1': [0.19496303796768188], 'k=2': [0.2080094814300537], 'k=3': [0.20657356083393097], 'k=4': [0.2166593223810196]}, 'pgu': {'k=1': [0.1554212123155594], 'k=2': [0.11097069829702377], 'k=3': [0.10050679743289948], 'k=4': [1.3307929869199597e-08]}}, 'sp_lime': {'pgi': {'k=1': [0.049393996596336365], 'k=2': [0.12943805754184723], 'k=3': [0.15528345108032227], 'k=4': [0.21556030213832855]}, 'pgu': {'k=1': [0.2171909362077713], 'k=2': [0.2058209478855133], 'k=3': [0.19505777955055237], 'k=4': [1.3307929869199597e-08]}}, 'pfi': {'pgi': {'k=1': [0.19496329128742218], 'k=2': [0.20834073424339294], 'k=3': [0.20540250837802887], 'k=4': [0.21636861562728882]}, 'pgu': {'k=1': [0.1546029895544052], 'k=2': [0.11036

In [6]:
auc_results,auc_show,sum_results,sum_show = aggregate_auc_or_sum(evaluation_metrics_per_sample,config)

# Print results nicely
for method, vals in auc_show.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']}, PGU AUC = {vals['pgu_auc']}")


shap: PGI AUC = 0.20087180342939168 +- 0.0014981356752109313, PGU AUC = 0.1174286470727788 +- 0.0012102381016898787
lime: PGI AUC = 0.20679806808630624 +- 0.0016253140962905446, PGU AUC = 0.09639603282014529 +- 0.0007411487271259391
sp_lime: PGI AUC = 0.1390662154638105 +- 0.001371466182046619, PGU AUC = 0.16982473250892427 +- 0.0012320390312792414
pfi: PGI AUC = 0.20646971940994266 +- 0.0016532002439635526, PGU AUC = 0.09598431014352377 +- 0.0007502488550474473
ig: PGI AUC = 0.20980021715164182 +- 0.001550731180675957, PGU AUC = 0.0791495736134756 +- 0.001119795943068326
sg: PGI AUC = 0.2099614021513197 +- 0.0015415080799846613, PGU AUC = 0.07928350388796793 +- 0.0011117206208845329
itg: PGI AUC = 0.20071117848157885 +- 0.0015044153044758183, PGU AUC = 0.11757074190510645 +- 0.0012173906568362348
icc_topo: PGI AUC = 0.2065737467010816 +- 0.0016288987205346351, PGU AUC = 0.09611538277732 +- 0.0007307946573111239
icc_shap: PGI AUC = 0.20709035164780085 +- 0.0016652616603875033, PGU AUC 

In [7]:
for method, vals in sum_show.items():
    print(f"{method}: PGI SUM = {vals['pgi_sum']}, PGU SUM = {vals['pgu_sum']}")

shap: PGI SUM = 0.8082056641578674 +- 0.006054817619377324, PGU SUM = 0.42974212765693665 +- 0.004195135181656868
lime: PGI SUM = 0.826205313205719 +- 0.006423072814231208, PGU SUM = 0.36689871549606323 +- 0.0027777243451784866
sp_lime: PGI SUM = 0.5496758222579956 +- 0.005527411029848834, PGU SUM = 0.618069589138031 +- 0.004354852634243401
pfi: PGI SUM = 0.8250751495361328 +- 0.006469566821853766, PGU SUM = 0.3652544319629669 +- 0.002794645857137835
ig: PGI SUM = 0.83408522605896 +- 0.006200288723537771, PGU SUM = 0.3149529695510864 +- 0.003921527850530308
sg: PGI SUM = 0.83480304479599 +- 0.006201021770054561, PGU SUM = 0.3153935372829437 +- 0.0038838407428443142
itg: PGI SUM = 0.8072425723075867 +- 0.006055576216063163, PGU SUM = 0.4304734170436859 +- 0.004206025332877039
icc_topo: PGI SUM = 0.8251115679740906 +- 0.006406349620250676, PGU SUM = 0.3658786714076996 +- 0.002750904576061727
icc_shap: PGI SUM = 0.8268226385116577 +- 0.006543757821063798, PGU SUM = 0.36591166257858276 +- 

## Lung Cancer Dataset

In [2]:
config = CONFIG['cancer']
mlp_path = PREFIX + 'models/cancer_mlp/cancer_mlp_seed_3/logs/version_0/checkpoints/epoch=39-step=4720.ckpt'
scm_path = PREFIX + 'models/cancer_nf/cancer_nf_seed_2/logs/version_0/checkpoints/epoch=58-step=6962.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [9]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [14]:
# Load JSON
with open(PREFIX +"attributions/cancer/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

In [11]:
ge_dict = data["ge_dict"]
evaluation_metrics,evaluation_metrics_per_sample = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)


{'shap': {'pgi': {'k=1': [0.2134249359369278], 'k=2': [0.29935744404792786], 'k=3': [0.3327498733997345], 'k=4': [0.3791840672492981], 'k=5': [0.4066338837146759], 'k=6': [0.41710737347602844], 'k=7': [0.46727004647254944]}, 'pgu': {'k=1': [0.4375058114528656], 'k=2': [0.4146712124347687], 'k=3': [0.41035526990890503], 'k=4': [0.37464219331741333], 'k=5': [0.3473282754421234], 'k=6': [0.29170188307762146], 'k=7': [1.732276899701901e-08]}}, 'lime': {'pgi': {'k=1': [0.29096922278404236], 'k=2': [0.30258649587631226], 'k=3': [0.353669136762619], 'k=4': [0.3920283615589142], 'k=5': [0.419399619102478], 'k=6': [0.43815872073173523], 'k=7': [0.46740299463272095]}, 'pgu': {'k=1': [0.4173593521118164], 'k=2': [0.40230390429496765], 'k=3': [0.387849897146225], 'k=4': [0.3360205590724945], 'k=5': [0.2688675820827484], 'k=6': [0.21395662426948547], 'k=7': [1.732276899701901e-08]}}, 'sp_lime': {'pgi': {'k=1': [0.1240033432841301], 'k=2': [0.2449956238269806], 'k=3': [0.29447701573371887], 'k=4': [

In [12]:
auc_results,auc_show,sum_results,sum_show = aggregate_auc_or_sum(evaluation_metrics_per_sample,config)

# Print results nicely
for method, vals in auc_show.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']}, PGU AUC = {vals['pgu_auc']}")


shap: PGI AUC = 0.3625633541329039 +- 0.005434405297235846, PGU AUC = 0.3429086207996847 +- 0.003786609893473744
lime: PGI AUC = 0.3808380662197144 +- 0.004233050825645996, PGU AUC = 0.30294636474404896 +- 0.0048819548680504475
sp_lime: PGI AUC = 0.3480917841608688 +- 0.004916477859525521, PGU AUC = 0.3105389919873819 +- 0.0038085373972594114
pfi: PGI AUC = 0.41602669455111024 +- 0.005130484205305648, PGU AUC = 0.27678879071744933 +- 0.004258752647857307
ig: PGI AUC = 0.422581837479439 +- 0.005494635877927833, PGU AUC = 0.25079258081322126 +- 0.0032929148033632965
sg: PGI AUC = 0.3589843003017207 +- 0.005542195933280121, PGU AUC = 0.3302222847351287 +- 0.0037773637735847966
itg: PGI AUC = 0.34762859535351803 +- 0.0054549638024604325, PGU AUC = 0.34224078832828025 +- 0.003782746217289304
icc_topo: PGI AUC = 0.42503153905065527 +- 0.005425998826980188, PGU AUC = 0.2543862634777583 +- 0.0033561569825395553
icc_shap: PGI AUC = 0.42044343039807347 +- 0.005222422671933947, PGU AUC = 0.264047

In [13]:
for method, vals in sum_show.items():
    print(f"{method}: PGI SUM = {vals['pgi_sum']}, PGU SUM = {vals['pgu_sum']}")

shap: PGI SUM = 2.5157275199890137 +- 0.03761018707705641, PGU SUM = 2.2762045860290527 +- 0.02577621504553081
lime: PGI SUM = 2.6642143726348877 +- 0.029630891182910676, PGU SUM = 2.026357889175415 +- 0.032575742833868854
sp_lime: PGI SUM = 2.3839352130889893 +- 0.033556874192084724, PGU SUM = 2.0917434692382812 +- 0.025993595883807546
pfi: PGI SUM = 2.8757758140563965 +- 0.03499480850602114, PGU SUM = 1.8692442178726196 +- 0.028746006094831144
ig: PGI SUM = 2.9147613048553467 +- 0.03718131167278216, PGU SUM = 1.7137739658355713 +- 0.02292105178798616
sg: PGI SUM = 2.49440860748291 +- 0.03826398426950129, PGU SUM = 2.2000572681427 +- 0.02565349150021045
itg: PGI SUM = 2.426779270172119 +- 0.03775068623998722, PGU SUM = 2.2721328735351562 +- 0.025723693631361903
icc_topo: PGI SUM = 2.929060220718384 +- 0.036736378595597384, PGU SUM = 1.7351601123809814 +- 0.02327675880814974
icc_shap: PGI SUM = 2.902144193649292 +- 0.03555577828428349, PGU SUM = 1.7927380800247192 +- 0.0274519046325491

## Synthetic Data

In [9]:
config = CONFIG['syn']
mlp_path = PREFIX + 'models/syn_mlp/syn_mlp_seed_1/logs/version_4/checkpoints/epoch=75-step=8968.ckpt'
scm_path = PREFIX + 'models/syn_nf/syn_nf_seed_1/logs/version_5/checkpoints/epoch=50-step=6018.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_mlp'],classification=config['classification']))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))

In [3]:
##generate attributions
ge_dict, time_dict, global_explanations = generate_global_exps(config,mlp_model.to('cpu'),scm_model.to('cpu'))

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [10]:
# Load JSON
with open(PREFIX +"attributions/syn/attribution.json", "r") as f:
    data = json.load(f)
global_explanations = data['global_explanations']    
generate_attr_plot(global_explanations,config)

In [11]:
ge_dict = data["ge_dict"]
evaluation_metrics,evaluation_metrics_per_sample = evaluate_exp(ge_dict,config,mlp_model.to('cpu'))
print(evaluation_metrics)


{'shap': {'pgi': {'k=1': [0.00820283591747284], 'k=2': [0.0243816077709198], 'k=3': [0.05584235489368439]}, 'pgu': {'k=1': [0.05084645003080368], 'k=2': [0.036981046199798584], 'k=3': [1.410643246657628e-08]}}, 'lime': {'pgi': {'k=1': [0.03696630150079727], 'k=2': [0.040460389107465744], 'k=3': [0.055902350693941116]}, 'pgu': {'k=1': [0.02444577030837536], 'k=2': [0.015607663430273533], 'k=3': [1.410643246657628e-08]}}, 'sp_lime': {'pgi': {'k=1': [0.008191952481865883], 'k=2': [0.04043359309434891], 'k=3': [0.055893540382385254]}, 'pgu': {'k=1': [0.050810545682907104], 'k=2': [0.015613099560141563], 'k=3': [1.410643246657628e-08]}}, 'pfi': {'pgi': {'k=1': [0.03700951859354973], 'k=2': [0.04042484611272812], 'k=3': [0.055911317467689514]}, 'pgu': {'k=1': [0.024393396452069283], 'k=2': [0.015585404820740223], 'k=3': [1.410643246657628e-08]}}, 'ig': {'pgi': {'k=1': [0.015590312890708447], 'k=2': [0.024419108405709267], 'k=3': [0.055896926671266556]}, 'pgu': {'k=1': [0.04048353433609009], 

In [12]:
auc_results,auc_show,sum_results,sum_show = aggregate_auc_or_sum(evaluation_metrics_per_sample,config)

# Print results nicely
for method, vals in auc_show.items():
    print(f"{method}: PGI AUC = {vals['pgi_auc']}, PGU AUC = {vals['pgu_auc']}")


shap: PGI AUC = 0.02820210188658287 +- 3.9031915729876e-05, PGU AUC = 0.031202139204988877 +- 2.7871654229467406e-05
lime: PGI AUC = 0.043447356641292574 +- 4.380799708438205e-05, PGU AUC = 0.013915278320976844 +- 2.9956640466214713e-05
sp_lime: PGI AUC = 0.036238169092684985 +- 4.2895265275679185e-05, PGU AUC = 0.020509190280611315 +- 3.345991697232605e-05
pfi: PGI AUC = 0.0434426319686075 +- 4.4184797321083684e-05, PGU AUC = 0.013891055805298189 +- 2.9710514444322555e-05
ig: PGI AUC = 0.030081364828472335 +- 4.1821093732860285e-05, PGU AUC = 0.028598410855357845 +- 2.8000463763153076e-05
sg: PGI AUC = 0.03007141756825149 +- 4.143222274114141e-05, PGU AUC = 0.02859446599831184 +- 2.8131021027048164e-05
itg: PGI AUC = 0.04346702910338839 +- 4.4839172586310005e-05, PGU AUC = 0.013888611432630569 +- 2.991858363098976e-05
icc_topo: PGI AUC = 0.04862377510095636 +- 4.230141616437587e-05, PGU AUC = 0.010206636927012975 +- 2.5285428056484958e-05
icc_shap: PGI AUC = 0.048595535981158414 +- 4.

In [13]:
for method, vals in sum_show.items():
    print(f"{method}: PGI SUM = {vals['pgi_sum']}, PGU SUM0= {vals['pgu_sum']}")

shap: PGI SUM = 0.08842679858207703 +- 0.00011286104288096126, PGU SUM0= 0.08782751113176346 +- 7.659825663988258e-05
lime: PGI SUM = 0.13332903385162354 +- 0.00012385037188691582, PGU SUM0= 0.04005345329642296 +- 7.822201238732559e-05
sp_lime: PGI SUM = 0.1045190840959549 +- 0.00011797956374373816, PGU SUM0= 0.06642366200685501 +- 9.250985582322124e-05
pfi: PGI SUM = 0.13334567844867706 +- 0.00012373823035573747, PGU SUM0= 0.039978817105293274 +- 7.757552433535884e-05
ig: PGI SUM = 0.09590635448694229 +- 0.00012841786386860518, PGU SUM0= 0.0774386003613472 +- 7.831632333088332e-05
sg: PGI SUM = 0.09586615115404129 +- 0.00012807750764853477, PGU SUM0= 0.07741102576255798 +- 7.823041668991954e-05
itg: PGI SUM = 0.13332217931747437 +- 0.000126362128170166, PGU SUM0= 0.03996911272406578 +- 7.782551927339096e-05
icc_topo: PGI SUM = 0.1436234414577484 +- 0.00012008736042963335, PGU SUM0= 0.03261693939566612 +- 7.16107659483056e-05
icc_shap: PGI SUM = 0.14356756210327148 +- 0.000121107359014

In [ ]:
##Misc
#WD dimensionwise

#Syn
P = [0.0709,0.0872,0.0870]
Q = [0.0119,0.0636,0.0192]
R = [0.0603,0.0481,0.0627] 

#German
gender = [0.0351,0.0460,0.0650]
age = [0.0545,0.0594,0.0183]
amount = [0.0505,0.0393,0.0707]
duration = [0.1791,0.1075,0.2515]

#Cancer
asia = [0.0443,0.0401,0.0515]
tub = [0.0351,0.0419,0.0401]
smoke = [0.0251,0.0191,0.0146]
lung = [0.0305,0.0313,0.0369]
bronc = [0.0499,0.0392,0.0455]
either = [0.0286,0.0285,0.0295]
xray  = [0.0646,0.0687,0.0633]


## Test

In [2]:
config = CONFIG['syn']
mlp_path = '/home/dhruv/Files/Thesis/Dissertation/Code/models/syn_ann/syn_ann_seed_10/logs/version_0/checkpoints/epoch=161-step=5184.ckpt'
scm_path = '/home/dhruv/Files/Thesis/Dissertation/Code/models/syn_nf/syn_nf_seed_10/logs/version_0/checkpoints/epoch=19-step=640.ckpt'
mlp_model = MLPLightning.load_from_checkpoint(mlp_path,model = MLP(config['num_features'],config['hidden_layers_ann'],classification=False))
scm_model = CausalNF.load_from_checkpoint(scm_path,flow = flow(config['num_features'],config['gd_adjacency'],config['hidden_layers_flow']))
ge_dict, evaluation_metrics, time_dict, global_explanations = generate_global_exps(config,mlp_model,scm_model)

Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1
Max Iterations:  100
Starting Iteration:  1


In [3]:
ge_dict

{'ig': {'W': [0.008685602410414437],
  'Z': [0.5541948194138167],
  'X': [0.4371195781757688]},
 'itg': {'W': [0.008485306748260027],
  'Z': [0.497316907775009],
  'X': [0.49419778547673104]},
 'sg': {'W': [0.13136424456458626],
  'Z': [0.3860659244300918],
  'X': [0.48256983100532197]},
 'shap': {'W': [0.5803365821763931],
  'Z': [0.3543153621348831],
  'X': [0.06534805568872387]},
 'lime': {'W': [0.2837432524861816],
  'Z': [0.293367905107146],
  'X': [0.4228888424066725]},
 'sp_lime': {'X': [0.41602805243460733],
  'W': [0.2947343644233361],
  'Z': [0.2892375831420565]},
 'pfi': {'W': [0.2555289566516876],
  'Z': [0.2728649377822876],
  'X': [0.47160604596138]},
 'icc_topo': {'W': [0.07992755621671677],
  'Z': [0.6083893179893494],
  'X': [0.31013962626457214]},
 'icc_shap': {'W': [0.1287594437599182],
  'Z': [0.6630550622940063],
  'X': [0.20664207637310028]}}

In [9]:
generate_attr_plot(global_explanations,config)

In [7]:
# Load JSON
with open("/home/dhruv/Files/Thesis/Dissertation/Code/attributions/syn/attribution.json", "r") as f:
    data = json.load(f)

# Call the function
summary = compute_pgu_pgi_sums(data["evaluation_metrics"])

# Display the results
for method, scores in summary.items():
    print(f"{method}: PGU Sum = {scores['pgu_sum']:.6f}, PGI Sum = {scores['pgi_sum']:.6f}")


ig: PGU Sum = 0.128419, PGI Sum = 0.000436
itg: PGU Sum = 0.128419, PGI Sum = 0.000436
sg: PGU Sum = 0.128418, PGI Sum = 0.000437
shap: PGU Sum = 0.128420, PGI Sum = 0.000435
lime: PGU Sum = 0.128418, PGI Sum = 0.000437
sp_lime: PGU Sum = 0.128420, PGI Sum = 0.000435
pfi: PGU Sum = 0.128418, PGI Sum = 0.000437
icc_topo: PGU Sum = 0.128419, PGI Sum = 0.000436
icc_shap: PGU Sum = 0.128419, PGI Sum = 0.000436
